# M2177.003100 Deep Learning <br> Assignment #3 Part 3: Transformer

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Jeonghee Jo, October 2019, and modified by Jungbeom Lee, October 2020.

This is about Transformer (Vaswani et al., 2017).
https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf

Original blog post & code:
https://github.com/Kyubyong/transformer (Tensorflow)
https://pytorch.org/tutorials/beginner/transformer_tutorial.html (PyTorch)


That said, you are allowed to copy paste the codes from the original repo.
HOWEVER, <font color=red> try to implement the model yourself first </font>, and consider the original source code as a last resort.
You will learn a lot while wrapping around your head during the implementation. And you will understand nuts and bolts of Transformers more clearly in a code level.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **all Assignment Part 1-3**, run the *CollectSubmission.sh* script with your **Student number** as input argument. <br>
This will produce a zipped file called *[Your student number].zip*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* 20xx-xxxxx)





### The Grading is as follows:

This assignment is an on/off one: just make this notebook **"work"** without problem by: 

1. Train your model using at least <font color=red> 12 different hyperparameter set </font>. Report performance results computed in the last code block <font color=red> for corresponding each hyperparameter set </font>. Plus, <font color=red> submit the one checkpoint file </font> of your best model. 

2. Please provide the analysis of changed hyper-parameters. (10 points)

The details are described in <font color=red>**transformer_modules.py**</font>. (There is nothing to implement in this notebook.)


Now proceed to the code.


### Hyper-parameters

In [68]:
# change hyper-parameters in this code block!


emsize = 512 # embedding dimension
nhid = 512 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 8 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
batch_size = 32
eval_batch_size = 16
epochs = 10 # The number of epochs

### Load and Batch Data

In [63]:
# pip install torchtext==0.6.0
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
%env CUDA_VISIBLE_DEVICES = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)



env: CUDA_VISIBLE_DEVICES=0


In [69]:
from transformer_modules import *
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [70]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [71]:
best_val_loss = float("inf")

best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()
torch.save(best_model.state_dict(), 'models/Transformer.pth')

| epoch   1 |   200/ 1863 batches | lr 5.00 | ms/batch 28.43 | loss  9.07 | ppl  8683.99
| epoch   1 |   400/ 1863 batches | lr 5.00 | ms/batch 28.33 | loss  7.21 | ppl  1356.57
| epoch   1 |   600/ 1863 batches | lr 5.00 | ms/batch 28.55 | loss  6.58 | ppl   722.39
| epoch   1 |   800/ 1863 batches | lr 5.00 | ms/batch 28.65 | loss  6.33 | ppl   563.24
| epoch   1 |  1000/ 1863 batches | lr 5.00 | ms/batch 28.96 | loss  6.19 | ppl   489.16
| epoch   1 |  1200/ 1863 batches | lr 5.00 | ms/batch 29.33 | loss  6.06 | ppl   428.93
| epoch   1 |  1400/ 1863 batches | lr 5.00 | ms/batch 30.50 | loss  5.94 | ppl   380.54
| epoch   1 |  1600/ 1863 batches | lr 5.00 | ms/batch 29.74 | loss  5.87 | ppl   354.29
| epoch   1 |  1800/ 1863 batches | lr 5.00 | ms/batch 29.84 | loss  5.85 | ppl   348.35
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 56.54s | valid loss  5.69 | valid ppl   295.23
-----------------------------------

| epoch   9 |   200/ 1863 batches | lr 3.15 | ms/batch 29.19 | loss  4.60 | ppl    99.34
| epoch   9 |   400/ 1863 batches | lr 3.15 | ms/batch 29.21 | loss  4.54 | ppl    93.60
| epoch   9 |   600/ 1863 batches | lr 3.15 | ms/batch 29.28 | loss  4.54 | ppl    93.74
| epoch   9 |   800/ 1863 batches | lr 3.15 | ms/batch 29.24 | loss  4.57 | ppl    96.48
| epoch   9 |  1000/ 1863 batches | lr 3.15 | ms/batch 29.14 | loss  4.57 | ppl    96.55
| epoch   9 |  1200/ 1863 batches | lr 3.15 | ms/batch 29.31 | loss  4.58 | ppl    97.34
| epoch   9 |  1400/ 1863 batches | lr 3.15 | ms/batch 29.18 | loss  4.46 | ppl    86.46
| epoch   9 |  1600/ 1863 batches | lr 3.15 | ms/batch 29.12 | loss  4.48 | ppl    87.82
| epoch   9 |  1800/ 1863 batches | lr 3.15 | ms/batch 29.24 | loss  4.51 | ppl    90.87
-----------------------------------------------------------------------------------------
| end of epoch   9 | time: 56.46s | valid loss  5.12 | valid ppl   167.17
-----------------------------------

In [72]:
best_model_loaded = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
best_model_loaded.load_state_dict(torch.load('models/Transformer.pth'), strict=True)
test_loss = evaluate(best_model_loaded, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  5.05 | test ppl   155.74


In [3]:
%%HTML
<style>
td{font-size: 12px}
</style>

## Analysis on each hyper-parameter.

The experiment results for each parameter set are summarized as table below.

|exp#| emsize | nhid | nlayers | nhead | dropout | test_loss |
|----|--------|------|---------|-------|---------|-----------|
|  1 | 256    | 128  | 2       | 2     | 0.5     | 5.58      |
|  2 | 256    | 128  | 2       | 2     | 0.2     | 5.34      |
|  3 | 256    | 128  | 2       | 4     | 0.2     | 5.32      |
|  4 | 256    | 256  | 2       | 4     | 0.2     | 5.26      |
|  5 | 256    | 256  | 4       | 4     | 0.2     | 5.11      |
|  6 | 256    | 256  | 8       | 4     | 0.2     | 6.93      |
|  7 | 256    | 256  | 4       | 8     | 0.2     | 5.10      |
|  8 | 256    | 512  | 4       | 8     | 0.2     | 5.05      |
|  9 | 256    | 512  | 4       | 8     | 0.3     | 5.14      |
| 10 | 256    | 512  | 4       | 8     | 0.1     | 5.07      |
| 11 | 270    | 512  | 6       | 8     | 0.1     | 5.10      |
| 12 | 512    | 512  | 4       | 8     | 0.1     | 5.09      |
| 13 | 512    | 512  | 4       | 8     | 0.2     | 5.05      |

As shown in table above, one can observe that drop out rate 0.5 is too big for our model. (Since I ran code for only 10 epoch, high drop out rate makes thd model be underfitted.) Also, with emsize 256 and hid size 256, nlayer of 8 can't converge. (The experiment number 6.) Generally, the higher emsize, the higher nhid and the higher nhead lead to the better test_loss. But with epoch 10, we can observe that too deep or too big model couldn't converge enough. The chekpoint of best model which shows the best test_loss is attached in submission zip file.